In [3]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, \
    average_precision_score
from sklearn.model_selection import train_test_split
import time
import os
from pathlib import Path
from skimage import io
import copy
from torch import optim, cuda
import pandas as pd
import glob
from collections import Counter
# Useful for examining network
from functools import reduce
from operator import __add__
# from torchsummary import summary
import seaborn as sns
import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)
from PIL import Image
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# Useful for examining network
from functools import reduce
from operator import __add__
from torchsummary import summary

# from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns

import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)

# Image manipulations
from PIL import Image

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt


class Multi_2D_CNN_block(nn.Module):
    def __init__(self, in_channels, num_kernel):
        super(Multi_2D_CNN_block, self).__init__()
        conv_block = BasicConv2d
        self.a = conv_block(in_channels, int(num_kernel / 3), kernel_size=(1, 1))

        self.b = nn.Sequential(
            conv_block(in_channels, int(num_kernel / 2), kernel_size=(1, 1)),
            conv_block(int(num_kernel / 2), int(num_kernel), kernel_size=(3, 3))
        )

        self.c = nn.Sequential(
            conv_block(in_channels, int(num_kernel / 3), kernel_size=(1, 1)),
            conv_block(int(num_kernel / 3), int(num_kernel / 2), kernel_size=(3, 3)),
            conv_block(int(num_kernel / 2), int(num_kernel), kernel_size=(3, 3))
        )
        self.out_channels = int(num_kernel / 3) + int(num_kernel) + int(num_kernel)
        # I get out_channels is total number of out_channels for a/b/c
        self.bn = nn.BatchNorm2d(self.out_channels)

    def get_out_channels(self):
        return self.out_channels

    def forward(self, x):
        branch1 = self.a(x)
        branch2 = self.b(x)
        branch3 = self.c(x)
        output = [branch1, branch2, branch3]
        return self.bn(torch.cat(output,
                                 1))  # BatchNorm across the concatenation of output channels from final layer of Branch 1/2/3
        # ,1 refers to the channel dimension


class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super(BasicConv2d, self).__init__()
        conv_padding = reduce(__add__, [(k // 2 + (k - 2 * (k // 2)) - 1, k // 2) for k in kernel_size[::-1]])
        self.pad = nn.ZeroPad2d(conv_padding)
        # ZeroPad2d Output: :math:`(N, C, H_{out}, W_{out})` H_{out} is H_{in} with the padding to be added to either side of height
        # ZeroPad2d(2) would add 2 to all 4 sides, ZeroPad2d((1,1,2,0)) would add 1 left, 1 right, 2 above, 0 below
        # n_output_features = floor((n_input_features + 2(paddingsize) - convkernel_size) / stride_size) + 1
        # above creates same padding
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.pad(x)
        x = self.conv(x)
        x = F.relu(x, inplace=True)
        x = self.bn(x)
        return x


class MyModel(nn.Module):

    def __init__(self, initial_kernel_num=64):
        super(MyModel, self).__init__()

        multi_2d_cnn = Multi_2D_CNN_block
        conv_block = BasicConv2d

        self.conv_1 = conv_block(1, 64, kernel_size=(7, 3), stride=(2, 1))

        self.multi_2d_cnn_1a = nn.Sequential(
            multi_2d_cnn(in_channels=64, num_kernel=initial_kernel_num),
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num),
            nn.MaxPool2d(kernel_size=(3, 1))
        )

        self.multi_2d_cnn_1b = nn.Sequential(
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num * 1.5),
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 1.5),
            nn.MaxPool2d(kernel_size=(3, 1))
        )
        self.multi_2d_cnn_1c = nn.Sequential(
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 2),
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 2),
            nn.MaxPool2d(kernel_size=(2, 1))
        )

        self.multi_2d_cnn_2a = nn.Sequential(
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 4),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2b = nn.Sequential(
            multi_2d_cnn(in_channels=597, num_kernel=initial_kernel_num * 5),
            multi_2d_cnn(in_channels=746, num_kernel=initial_kernel_num * 6),
            multi_2d_cnn(in_channels=896, num_kernel=initial_kernel_num * 7),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2c = nn.Sequential(
            multi_2d_cnn(in_channels=1045, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2d = nn.Sequential(
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 12),
            multi_2d_cnn(in_channels=1792, num_kernel=initial_kernel_num * 14),
            multi_2d_cnn(in_channels=2090, num_kernel=initial_kernel_num * 16),
        )
        self.output = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(2389, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_1(x)
        # N x 1250 x 12 x 64 tensor
        x = self.multi_2d_cnn_1a(x)
        # N x 416 x 12 x 149 tensor
        x = self.multi_2d_cnn_1b(x)
        # N x 138 x 12 x 224 tensor
        x = self.multi_2d_cnn_1c(x)
        # N x 69 x 12 x 298
        x = self.multi_2d_cnn_2a(x)

        x = self.multi_2d_cnn_2b(x)

        x = self.multi_2d_cnn_2c(x)

        x = self.multi_2d_cnn_2d(x)

        x = self.output(x)

        return x



In [8]:

model = MyModel()

# Device check
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f'Train on: {device}')

Train on: cpu


In [7]:
summary(model, input_size=(1, 2500, 12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 1, 2506, 14]               0
            Conv2d-2         [-1, 64, 1250, 12]           1,344
       BatchNorm2d-3         [-1, 64, 1250, 12]             128
       BasicConv2d-4         [-1, 64, 1250, 12]               0
         ZeroPad2d-5         [-1, 64, 1250, 12]               0
            Conv2d-6         [-1, 21, 1250, 12]           1,344
       BatchNorm2d-7         [-1, 21, 1250, 12]              42
       BasicConv2d-8         [-1, 21, 1250, 12]               0
         ZeroPad2d-9         [-1, 64, 1250, 12]               0
           Conv2d-10         [-1, 32, 1250, 12]           2,048
      BatchNorm2d-11         [-1, 32, 1250, 12]              64
      BasicConv2d-12         [-1, 32, 1250, 12]               0
        ZeroPad2d-13         [-1, 32, 1252, 14]               0
           Conv2d-14         [-1, 64, 1

In [7]:
summary(model, input_size=(1, 2500, 12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 1, 2506, 14]               0
            Conv2d-2         [-1, 64, 1250, 12]           1,344
       BatchNorm2d-3         [-1, 64, 1250, 12]             128
       BasicConv2d-4         [-1, 64, 1250, 12]               0
         ZeroPad2d-5         [-1, 64, 1250, 12]               0
            Conv2d-6         [-1, 21, 1250, 12]           1,344
       BatchNorm2d-7         [-1, 21, 1250, 12]              42
       BasicConv2d-8         [-1, 21, 1250, 12]               0
         ZeroPad2d-9         [-1, 64, 1250, 12]               0
           Conv2d-10         [-1, 32, 1250, 12]           2,048
      BatchNorm2d-11         [-1, 32, 1250, 12]              64
      BasicConv2d-12         [-1, 32, 1250, 12]               0
        ZeroPad2d-13         [-1, 32, 1252, 14]               0
           Conv2d-14         [-1, 64, 1

In [9]:
class Inception_Tabular_7_1(nn.Module):

    def __init__(self, initial_kernel_num=64):
        super(Inception_Tabular_7_1, self).__init__()

        multi_2d_cnn = Multi_2D_CNN_block
        conv_block = BasicConv2d

        self.conv_1 = conv_block(1, 64, kernel_size=(7, 1), stride=(2, 1))

        self.multi_2d_cnn_1a = nn.Sequential(
            multi_2d_cnn(in_channels=64, num_kernel=initial_kernel_num),
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num),
            nn.MaxPool2d(kernel_size=(3, 1))
        )

        self.multi_2d_cnn_1b = nn.Sequential(
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num * 1.5),
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 1.5),
            nn.MaxPool2d(kernel_size=(3, 1))
        )
        self.multi_2d_cnn_1c = nn.Sequential(
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 2),
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 2),
            nn.MaxPool2d(kernel_size=(2, 1))
        )

        self.multi_2d_cnn_2a = nn.Sequential(
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 4),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2b = nn.Sequential(
            multi_2d_cnn(in_channels=597, num_kernel=initial_kernel_num * 5),
            multi_2d_cnn(in_channels=746, num_kernel=initial_kernel_num * 6),
            multi_2d_cnn(in_channels=896, num_kernel=initial_kernel_num * 7),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2c = nn.Sequential(
            multi_2d_cnn(in_channels=1045, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2d = nn.Sequential(
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 12),
            multi_2d_cnn(in_channels=1792, num_kernel=initial_kernel_num * 14),
            multi_2d_cnn(in_channels=2090, num_kernel=initial_kernel_num * 16),
        )
        self.output = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(2389, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_1(x)
        # N x 1250 x 12 x 64 tensor
        x = self.multi_2d_cnn_1a(x)
        # N x 416 x 12 x 149 tensor
        x = self.multi_2d_cnn_1b(x)
        # N x 138 x 12 x 224 tensor
        x = self.multi_2d_cnn_1c(x)
        # N x 69 x 12 x 298
        x = self.multi_2d_cnn_2a(x)

        x = self.multi_2d_cnn_2b(x)

        x = self.multi_2d_cnn_2c(x)

        x = self.multi_2d_cnn_2d(x)

        x = self.output(x)

        return x

class Inception_Tabular_15_3(nn.Module):

    def __init__(self, initial_kernel_num=64):
        super(Inception_Tabular_15_3, self).__init__()

        multi_2d_cnn = Multi_2D_CNN_block
        conv_block = BasicConv2d

        self.conv_1 = conv_block(1, 64, kernel_size=(15, 3), stride=(2, 1))

        self.multi_2d_cnn_1a = nn.Sequential(
            multi_2d_cnn(in_channels=64, num_kernel=initial_kernel_num),
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num),
            nn.MaxPool2d(kernel_size=(3, 1))
        )

        self.multi_2d_cnn_1b = nn.Sequential(
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num * 1.5),
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 1.5),
            nn.MaxPool2d(kernel_size=(3, 1))
        )
        self.multi_2d_cnn_1c = nn.Sequential(
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 2),
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 2),
            nn.MaxPool2d(kernel_size=(2, 1))
        )

        self.multi_2d_cnn_2a = nn.Sequential(
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 4),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2b = nn.Sequential(
            multi_2d_cnn(in_channels=597, num_kernel=initial_kernel_num * 5),
            multi_2d_cnn(in_channels=746, num_kernel=initial_kernel_num * 6),
            multi_2d_cnn(in_channels=896, num_kernel=initial_kernel_num * 7),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2c = nn.Sequential(
            multi_2d_cnn(in_channels=1045, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2d = nn.Sequential(
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 12),
            multi_2d_cnn(in_channels=1792, num_kernel=initial_kernel_num * 14),
            multi_2d_cnn(in_channels=2090, num_kernel=initial_kernel_num * 16),
        )
        self.output = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(2389, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_1(x)
        # N x 1250 x 12 x 64 tensor
        x = self.multi_2d_cnn_1a(x)
        # N x 416 x 12 x 149 tensor
        x = self.multi_2d_cnn_1b(x)
        # N x 138 x 12 x 224 tensor
        x = self.multi_2d_cnn_1c(x)
        # N x 69 x 12 x 298
        x = self.multi_2d_cnn_2a(x)

        x = self.multi_2d_cnn_2b(x)

        x = self.multi_2d_cnn_2c(x)

        x = self.multi_2d_cnn_2d(x)

        x = self.output(x)

        return x


class Inception_Tabular_15_1(nn.Module):

    def __init__(self, initial_kernel_num=64):
        super(Inception_Tabular_15_1, self).__init__()

        multi_2d_cnn = Multi_2D_CNN_block
        conv_block = BasicConv2d

        self.conv_1 = conv_block(1, 64, kernel_size=(15, 1), stride=(2, 1))

        self.multi_2d_cnn_1a = nn.Sequential(
            multi_2d_cnn(in_channels=64, num_kernel=initial_kernel_num),
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num),
            nn.MaxPool2d(kernel_size=(3, 1))
        )

        self.multi_2d_cnn_1b = nn.Sequential(
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num * 1.5),
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 1.5),
            nn.MaxPool2d(kernel_size=(3, 1))
        )
        self.multi_2d_cnn_1c = nn.Sequential(
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 2),
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 2),
            nn.MaxPool2d(kernel_size=(2, 1))
        )

        self.multi_2d_cnn_2a = nn.Sequential(
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 4),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2b = nn.Sequential(
            multi_2d_cnn(in_channels=597, num_kernel=initial_kernel_num * 5),
            multi_2d_cnn(in_channels=746, num_kernel=initial_kernel_num * 6),
            multi_2d_cnn(in_channels=896, num_kernel=initial_kernel_num * 7),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2c = nn.Sequential(
            multi_2d_cnn(in_channels=1045, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2d = nn.Sequential(
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 12),
            multi_2d_cnn(in_channels=1792, num_kernel=initial_kernel_num * 14),
            multi_2d_cnn(in_channels=2090, num_kernel=initial_kernel_num * 16),
        )
        self.output = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(2389, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_1(x)
        # N x 1250 x 12 x 64 tensor
        x = self.multi_2d_cnn_1a(x)
        # N x 416 x 12 x 149 tensor
        x = self.multi_2d_cnn_1b(x)
        # N x 138 x 12 x 224 tensor
        x = self.multi_2d_cnn_1c(x)
        # N x 69 x 12 x 298
        x = self.multi_2d_cnn_2a(x)

        x = self.multi_2d_cnn_2b(x)

        x = self.multi_2d_cnn_2c(x)

        x = self.multi_2d_cnn_2d(x)

        x = self.output(x)

        return x

class Inception_Tabular_5_Linears(nn.Module):

    def __init__(self, initial_kernel_num=64):
        super(Inception_Tabular_5_Linears, self).__init__()

        multi_2d_cnn = Multi_2D_CNN_block
        conv_block = BasicConv2d

        self.conv_1 = conv_block(1, 64, kernel_size=(7, 3), stride=(2, 1))

        self.multi_2d_cnn_1a = nn.Sequential(
            multi_2d_cnn(in_channels=64, num_kernel=initial_kernel_num),
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num),
            nn.MaxPool2d(kernel_size=(3, 1))
        )

        self.multi_2d_cnn_1b = nn.Sequential(
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num * 1.5),
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 1.5),
            nn.MaxPool2d(kernel_size=(3, 1))
        )
        self.multi_2d_cnn_1c = nn.Sequential(
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 2),
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 2),
            nn.MaxPool2d(kernel_size=(2, 1))
        )

        self.multi_2d_cnn_2a = nn.Sequential(
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 4),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2b = nn.Sequential(
            multi_2d_cnn(in_channels=597, num_kernel=initial_kernel_num * 5),
            multi_2d_cnn(in_channels=746, num_kernel=initial_kernel_num * 6),
            multi_2d_cnn(in_channels=896, num_kernel=initial_kernel_num * 7),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2c = nn.Sequential(
            multi_2d_cnn(in_channels=1045, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2d = nn.Sequential(
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 12),
            multi_2d_cnn(in_channels=1792, num_kernel=initial_kernel_num * 14),
            multi_2d_cnn(in_channels=2090, num_kernel=initial_kernel_num * 16),
        )
        self.output = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(2389, 1200),
            nn.Linear(1200, 400),
            nn.Linear(400, 100),
            nn.Linear(100, 20),
            nn.Linear(20, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_1(x)
        # N x 1250 x 12 x 64 tensor
        x = self.multi_2d_cnn_1a(x)
        # N x 416 x 12 x 149 tensor
        x = self.multi_2d_cnn_1b(x)
        # N x 138 x 12 x 224 tensor
        x = self.multi_2d_cnn_1c(x)
        # N x 69 x 12 x 298
        x = self.multi_2d_cnn_2a(x)

        x = self.multi_2d_cnn_2b(x)

        x = self.multi_2d_cnn_2c(x)

        x = self.multi_2d_cnn_2d(x)

        x = self.output(x)

        return x

class Inception_Tabular_5_Linears_BatchNorm(nn.Module):

    def __init__(self, initial_kernel_num=64):
        super(Inception_Tabular_5_Linears, self).__init__()

        multi_2d_cnn = Multi_2D_CNN_block
        conv_block = BasicConv2d

        self.conv_1 = conv_block(1, 64, kernel_size=(7, 3), stride=(2, 1))

        self.multi_2d_cnn_1a = nn.Sequential(
            multi_2d_cnn(in_channels=64, num_kernel=initial_kernel_num),
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num),
            nn.MaxPool2d(kernel_size=(3, 1))
        )

        self.multi_2d_cnn_1b = nn.Sequential(
            multi_2d_cnn(in_channels=149, num_kernel=initial_kernel_num * 1.5),
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 1.5),
            nn.MaxPool2d(kernel_size=(3, 1))
        )
        self.multi_2d_cnn_1c = nn.Sequential(
            multi_2d_cnn(in_channels=224, num_kernel=initial_kernel_num * 2),
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 2),
            nn.MaxPool2d(kernel_size=(2, 1))
        )

        self.multi_2d_cnn_2a = nn.Sequential(
            multi_2d_cnn(in_channels=298, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 3),
            multi_2d_cnn(in_channels=448, num_kernel=initial_kernel_num * 4),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2b = nn.Sequential(
            multi_2d_cnn(in_channels=597, num_kernel=initial_kernel_num * 5),
            multi_2d_cnn(in_channels=746, num_kernel=initial_kernel_num * 6),
            multi_2d_cnn(in_channels=896, num_kernel=initial_kernel_num * 7),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2c = nn.Sequential(
            multi_2d_cnn(in_channels=1045, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 8),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.multi_2d_cnn_2d = nn.Sequential(
            multi_2d_cnn(in_channels=1194, num_kernel=initial_kernel_num * 12),
            multi_2d_cnn(in_channels=1792, num_kernel=initial_kernel_num * 14),
            multi_2d_cnn(in_channels=2090, num_kernel=initial_kernel_num * 16),
        )
        self.output = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(2389, 1200),
            nn.BatchNorm1d(1200),
            nn.Linear(1200, 400),
            nn.BatchNorm1d(400),
            nn.Linear(400, 100),
            nn.BatchNorm1d(100),
            nn.Linear(100, 20),
            nn.BatchNorm1d(20),
            nn.Linear(20, 1)
            # nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_1(x)
        # N x 1250 x 12 x 64 tensor
        x = self.multi_2d_cnn_1a(x)
        # N x 416 x 12 x 149 tensor
        x = self.multi_2d_cnn_1b(x)
        # N x 138 x 12 x 224 tensor
        x = self.multi_2d_cnn_1c(x)
        # N x 69 x 12 x 298
        x = self.multi_2d_cnn_2a(x)

        x = self.multi_2d_cnn_2b(x)

        x = self.multi_2d_cnn_2c(x)

        x = self.multi_2d_cnn_2d(x)

        x = self.output(x)

        return x
    


In [10]:
model = Inception_Tabular_15_3()

In [12]:
summary(Inception_Tabular_15_3(), input_size=(1, 2500, 12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 1, 2514, 14]               0
            Conv2d-2         [-1, 64, 1250, 12]           2,880
       BatchNorm2d-3         [-1, 64, 1250, 12]             128
       BasicConv2d-4         [-1, 64, 1250, 12]               0
         ZeroPad2d-5         [-1, 64, 1250, 12]               0
            Conv2d-6         [-1, 21, 1250, 12]           1,344
       BatchNorm2d-7         [-1, 21, 1250, 12]              42
       BasicConv2d-8         [-1, 21, 1250, 12]               0
         ZeroPad2d-9         [-1, 64, 1250, 12]               0
           Conv2d-10         [-1, 32, 1250, 12]           2,048
      BatchNorm2d-11         [-1, 32, 1250, 12]              64
      BasicConv2d-12         [-1, 32, 1250, 12]               0
        ZeroPad2d-13         [-1, 32, 1252, 14]               0
           Conv2d-14         [-1, 64, 1

In [13]:
summary(MyModel(), input_size=(1, 2500, 12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 1, 2506, 14]               0
            Conv2d-2         [-1, 64, 1250, 12]           1,344
       BatchNorm2d-3         [-1, 64, 1250, 12]             128
       BasicConv2d-4         [-1, 64, 1250, 12]               0
         ZeroPad2d-5         [-1, 64, 1250, 12]               0
            Conv2d-6         [-1, 21, 1250, 12]           1,344
       BatchNorm2d-7         [-1, 21, 1250, 12]              42
       BasicConv2d-8         [-1, 21, 1250, 12]               0
         ZeroPad2d-9         [-1, 64, 1250, 12]               0
           Conv2d-10         [-1, 32, 1250, 12]           2,048
      BatchNorm2d-11         [-1, 32, 1250, 12]              64
      BasicConv2d-12         [-1, 32, 1250, 12]               0
        ZeroPad2d-13         [-1, 32, 1252, 14]               0
           Conv2d-14         [-1, 64, 1

In [14]:
summary(Inception_Tabular_15_1(), input_size=(1, 2500, 12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 1, 2514, 12]               0
            Conv2d-2         [-1, 64, 1250, 12]             960
       BatchNorm2d-3         [-1, 64, 1250, 12]             128
       BasicConv2d-4         [-1, 64, 1250, 12]               0
         ZeroPad2d-5         [-1, 64, 1250, 12]               0
            Conv2d-6         [-1, 21, 1250, 12]           1,344
       BatchNorm2d-7         [-1, 21, 1250, 12]              42
       BasicConv2d-8         [-1, 21, 1250, 12]               0
         ZeroPad2d-9         [-1, 64, 1250, 12]               0
           Conv2d-10         [-1, 32, 1250, 12]           2,048
      BatchNorm2d-11         [-1, 32, 1250, 12]              64
      BasicConv2d-12         [-1, 32, 1250, 12]               0
        ZeroPad2d-13         [-1, 32, 1252, 14]               0
           Conv2d-14         [-1, 64, 1

In [89]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, \
    average_precision_score
from sklearn.model_selection import train_test_split
import time
import os
from pathlib import Path
from skimage import io
import copy
from torch import optim, cuda
import pandas as pd
import glob
from collections import Counter
# Useful for examining network
from functools import reduce
from operator import __add__
# from torchsummary import summary
import seaborn as sns
import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)
from PIL import Image
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# Useful for examining network
from functools import reduce
from operator import __add__
from torchsummary import summary

# from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns

import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)

# Image manipulations
from PIL import Image

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
class Net(nn.Module):

    # Convolution as a whole should span at least 1 beat, preferably more
    # Input shape = (1,2500,12)
#     summary(model, input_size=(1, 2500, 12))
    # CLEAR EXPERIMENTS TO TRY
#     Alter kernel size, right now drops to 10 channels at beginning then stays there
#     Try increasing output channel size as you go deeper 
#     Alter stride to have larger image at FC layer

    def __init__(self):
        super(Net, self).__init__()

        base_conv = 64
        self.conv1 = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=base_conv, kernel_size=(7,3), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1))
            )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1))
            )

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1))
            )
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.fc1 = nn.Sequential(
#             nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64*87*10, 64), #64 kernel size, 2500 pooled to 29, 
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1))

    def forward(self, x):
        out = self.conv1(x)
#         print(out.shape)
        out = self.conv2(out)
#         print(out.shape)        
        out = self.conv3(out)
#         print(out.shape)
        out = self.conv4(out)
#         print(out.shape)
        out = self.conv5(out)
#         print(out.shape)
        out = self.conv6(out)
#         print(out.shape)
        out = self.conv7(out)
#         print(out.shape)
        out = self.fc1(out)
        return out

model = Net()
print(model)


Net(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()


In [90]:
summary(model, input_size=(1, 2500, 12))

torch.Size([2, 64, 831, 10])
torch.Size([2, 64, 829, 10])
torch.Size([2, 64, 276, 10])
torch.Size([2, 64, 274, 10])
torch.Size([2, 64, 91, 10])
torch.Size([2, 64, 89, 10])
torch.Size([2, 64, 87, 10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 2494, 10]           1,408
              ReLU-2         [-1, 64, 2494, 10]               0
       BatchNorm2d-3         [-1, 64, 2494, 10]             128
            Conv2d-4         [-1, 64, 2494, 10]           4,160
              ReLU-5         [-1, 64, 2494, 10]               0
       BatchNorm2d-6         [-1, 64, 2494, 10]             128
         MaxPool2d-7          [-1, 64, 831, 10]               0
            Conv2d-8          [-1, 64, 831, 10]           4,160
              ReLU-9          [-1, 64, 831, 10]               0
      BatchNorm2d-10          [-1, 64, 831, 10]             128
           Conv2d-11          [

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, \
    average_precision_score
from sklearn.model_selection import train_test_split
import time
import os
from pathlib import Path
from skimage import io
import copy
from torch import optim, cuda
import pandas as pd
import glob
from collections import Counter
# Useful for examining network
from functools import reduce
from operator import __add__
# from torchsummary import summary
import seaborn as sns
import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)
from PIL import Image
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# Useful for examining network
from functools import reduce
from operator import __add__
from torchsummary import summary

# from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns

import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)

# Image manipulations
from PIL import Image

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
class Net(nn.Module):

    # Convolution as a whole should span at least 1 beat, preferably more
    # Input shape = (1,2500,12)
#     summary(model, input_size=(1, 2500, 12))
    # CLEAR EXPERIMENTS TO TRY
#     Alter kernel size, right now drops to 10 channels at beginning then stays there
#     Try increasing output channel size as you go deeper 
#     Alter stride to have larger image at FC layer

    def __init__(self):
        super(Net, self).__init__()

        base_conv = 64
        self.conv1 = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=base_conv, kernel_size=(7,3), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1))
            )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1))
            )

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1))
            )
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(3,1), stride=(1,1))
            )

        self.fc1 = nn.Sequential(
#             nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64*87*10, 64), #64 kernel size, 2500 pooled to 29, 
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1))

    def forward(self, x):
        out = self.conv1(x)
#         print(out.shape)
        out = self.conv2(out)
#         print(out.shape)        
        out = self.conv3(out)
#         print(out.shape)
        out = self.conv4(out)
#         print(out.shape)
        out = self.conv5(out)
#         print(out.shape)
        out = self.conv6(out)
#         print(out.shape)
        out = self.conv7(out)
#         print(out.shape)
        out = self.fc1(out)
        return out

model = Net()
print(model)


Net(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()


In [109]:
import matplotlib.pyplot as plt
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        base_conv = 16
        self.conv1 = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=base_conv, kernel_size=(5,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)) 
            )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(5,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)) 
            )

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv, out_channels=base_conv*2, kernel_size=(5,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*2),
            nn.Conv2d(in_channels=base_conv*2, out_channels=base_conv*2, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*2),
            nn.MaxPool2d(kernel_size=(4,1), stride=(4,1)) 
            )

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv*2, out_channels=base_conv*2, kernel_size=(3,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*2),
            nn.Conv2d(in_channels=base_conv*2, out_channels=base_conv*2, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*2),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)) 
            )

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv*2, out_channels=base_conv*4, kernel_size=(3,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*4),
            nn.Conv2d(in_channels=base_conv*4, out_channels=base_conv*4, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*4),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)) 
            )
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv*4, out_channels=base_conv*4, kernel_size=(3,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*4),
            nn.Conv2d(in_channels=base_conv*4, out_channels=base_conv*4, kernel_size=(1,1), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*4),
            nn.MaxPool2d(kernel_size=(4,1), stride=(4,1)) 
            )

        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=base_conv*4, out_channels=base_conv*4, kernel_size=(1,12), stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(base_conv*4),
            ) #Mayo describess using their 7th block as a spatial fusion block

        self.fc1 = nn.Sequential(
#             nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64*8*1, 64), #64 kernel size, #2500 decreased to 8, x1 lead
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
#             nn.Dropout(0.5),
            nn.Linear(32, 1))

    def forward(self, x):
        out = self.conv1(x)
#         print(out.shape)
        out = self.conv2(out)
#         print(out.shape)        
        out = self.conv3(out)
#         print(out.shape)
        out = self.conv4(out)
#         print(out.shape)
        out = self.conv5(out)
#         print(out.shape)
        out = self.conv6(out)
#         print(out.shape)
        out = self.conv7(out)
#         print(out.shape)
        out = self.fc1(out)
        return out

model = Net()
print(model)

Net(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 1), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 1), stride=(1, 1))
    (1): ReLU()


In [110]:
summary(model, input_size=(1, 2500, 12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 2496, 12]              96
              ReLU-2         [-1, 16, 2496, 12]               0
       BatchNorm2d-3         [-1, 16, 2496, 12]              32
            Conv2d-4         [-1, 16, 2496, 12]             272
              ReLU-5         [-1, 16, 2496, 12]               0
       BatchNorm2d-6         [-1, 16, 2496, 12]              32
         MaxPool2d-7         [-1, 16, 1248, 12]               0
            Conv2d-8         [-1, 16, 1244, 12]           1,296
              ReLU-9         [-1, 16, 1244, 12]               0
      BatchNorm2d-10         [-1, 16, 1244, 12]              32
           Conv2d-11         [-1, 16, 1244, 12]             272
             ReLU-12         [-1, 16, 1244, 12]               0
      BatchNorm2d-13         [-1, 16, 1244, 12]              32
        MaxPool2d-14          [-1, 16, 